# Explicación Paso a Paso: Generación de Rutas

Este notebook detalla el funcionamiento interno del algoritmo `generar_rutas` utilizado en `PathPlannerTabManager`. Se desglosan las etapas de carga de datos, procesamiento geométrico, generación de grafos y cálculo de trayectorias. Es un algoritmo de Planificación Probabilística de Hoja de Ruta (Probabilistic RoadMap - PRM) simplificado y determinista, adaptado a estructuras viales. En lugar de tirar puntos al azar, usa la estructura de "Calles" definida por el usuario como columna vertebral y proyecta extensiones hacia los objetivos (pozos), validando footprint y colisiones en cada paso.

In [1]:
import sys
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon

# Configuración del Path para importar módulos del proyecto
# Asumimos que este notebook está en src/core/op_blasting_gui/doc/jupyter/generar_rutas/
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../../src/enaex_gui"))
if project_root not in sys.path:
    sys.path.append(project_root)

import utils
import route_gen_logic
from fit_streets import fit_all_streets

print("Librerías importadas y path configurado: ", project_root)

ModuleNotFoundError: No module named 'utils'

## 1. Carga de Datos
Cargamos los archivos de entrada (pozos, geofence, calles, etc.) desde la carpeta de prueba.

In [ ]:
data_dir = "test_data/2_algorithm_explanation"

# Cargar Pozos (.hol)
holes_path = os.path.join(data_dir, "holes.hol")
# WGS84 = True (Simulando configuración por defecto)
holes = utils.readHolFile(holes_path, True)
print(f"Cargados {len(holes)} pozos.")

# Cargar Geometrías (.geojson)
geofence = gpd.read_file(os.path.join(data_dir, "geofence.geojson")).to_crs(holes.crs)
home_pose = gpd.read_file(os.path.join(data_dir, "home_pose.geojson")).to_crs(holes.crs)
streets = gpd.read_file(os.path.join(data_dir, "streets.geojson")).to_crs(holes.crs)

# Archivos opcionales
try:
    transit_streets = gpd.read_file(os.path.join(data_dir, "transit_streets.geojson")).to_crs(holes.crs)
except:
    transit_streets = None


try:
    obstacles = gpd.read_file(os.path.join(data_dir, "obstacles.geojson")).to_crs(holes.crs)
except:
    obstacles = None

try:
    high_obstacles = gpd.read_file(os.path.join(data_dir, "high_obstacles.geojson")).to_crs(holes.crs)
except:
    high_obstacles = None
print("Geometrías cargadas.")

## 2. Visualización Inicial
Visualizamos los datos crudos para entender el escenario.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
geofence.boundary.plot(ax=ax, color='green', label='Geofence')
holes.plot(ax=ax, color='black', markersize=5, label='Holes')
streets.plot(ax=ax, color='blue', label='Streets')
home_pose.plot(ax=ax, color='red', label='Home')
if obstacles is not None:
    obstacles.plot(ax=ax, color='red', alpha=0.5, label='Obstacles')
if obstacles is not None:
    high_obstacles.plot(ax=ax, color='red', alpha=0.5, label='High Obstacles')
plt.legend()
plt.title("Datos Iniciales")
plt.show()

## 3. Ajuste de Calles (Fit Streets)
El algoritmo ajusta las calles dibujadas para que pasen de manera óptima entre las filas de pozos.

In [ ]:
fit_streets_enabled = True
fit_twice = True

streets_fitted = streets.copy()
if fit_streets_enabled:
    streets_fitted = fit_all_streets(streets, holes, geofence, obstacles, None, fit_twice)

fig, ax = plt.subplots(figsize=(10, 10))
streets.plot(ax=ax, color='blue', alpha=0.5, label='Original Streets')
streets_fitted.plot(ax=ax, color='cyan', linestyle='--', label='Fitted Streets')
holes.plot(ax=ax, color='black', markersize=5)
plt.legend()
plt.title("Ajuste de Calles")
plt.show()

## 4. Filtrado y Definición de Zonas Bloqueadas
Se eliminan los pozos fuera del geofence o dentro de obstáculos, y se define el área 'blocked' (prohibida) para el robot.

In [ ]:
OBSTACLE_BUFFER_DISTANCE = 0.75

# Definir zona bloqueada (Borde Geofence + Obstáculos)
blocked = gpd.GeoDataFrame(geometry=gpd.GeoSeries(geofence.boundary)).buffer(OBSTACLE_BUFFER_DISTANCE).unary_union
holes_filtered = holes[holes.within(geofence.unary_union)].reset_index(drop=True)

if obstacles is not None:
    blocked = blocked.union(obstacles.buffer(OBSTACLE_BUFFER_DISTANCE).unary_union)
    holes_filtered = holes_filtered[holes_filtered.within(geofence.unary_union - obstacles.unary_union)].reset_index(drop=True)

print(f"Pozos después del filtrado: {len(holes_filtered)}")

## 5. Asociación de Pozos a Calles
Para cada pozo, identificamos cuál es la calle más cercana desde donde se intentará el abordaje.

In [ ]:
closest_streets = []
for i in range(0, len(holes_filtered)):
    distance = 10000.0
    closest_street = -1
    for j in range(0, len(streets_fitted)):
        dist = holes_filtered['geometry'][i].distance(streets_fitted['geometry'][j])
        if dist < distance:
            distance = dist
            closest_street = j
    closest_streets.append(closest_street)
    
holes_filtered['closest_street'] = closest_streets

## 6. Generación de Poses de Calle (Nodos del Grafo)
Discretizamos las líneas de las calles en puntos (poses) navegables.

In [ ]:
# Generar Poses de Calle
poses_street = utils.posesFromGeoDataFrame(streets_fitted, blocked, obstacles, None, geofence)
print(f"Generadas {len(poses_street)} poses de calle.")

# Visualizar
street_points = [Point(p[0], p[1]) for p in poses_street]
fig, ax = plt.subplots(figsize=(10, 10))
geofence.boundary.plot(ax=ax, color='green')
gpd.GeoSeries(street_points).plot(ax=ax, markersize=2, color='blue', label='Street Poses')
plt.title("Poses de Calle Generadas")
plt.legend()
plt.show()

## 7. Generación de Poses de Carguío (Loading Poses)
Esta es la etapa crítica: se calculan las trayectorias curvas desde la calle hasta cada pozo.

In [ ]:
# Mock para el callback de progreso
class MockSignal:
    def emit(self, val):
        pass

TURNING_RADIUS = 3.0
HOLE_DISTANCE = 2.5 + 1.3

# Preparar home pose (simplificado)
# Asumimos que home_pose tiene una geometría válida
home_coords = list(home_pose.geometry[0].coords)
import math
home_angle = math.atan2(home_coords[1][1]-home_coords[0][1], home_coords[1][0]-home_coords[0][0])
home_pose_0 = [[home_coords[0][0], home_coords[0][1], home_angle]]

prev_poses = home_pose_0 + poses_street

# Generar Loading Poses
poses_holes, extra_connections = utils.generateLoadingPoses(
    streets_fitted, holes_filtered, blocked, prev_poses, 
    OBSTACLE_BUFFER_DISTANCE, TURNING_RADIUS, HOLE_DISTANCE, MockSignal()
)

print(f"Generadas poses de carguío para {len(poses_holes)} pozos.")

## 8. Visualización Final de Rutas
Mostramos las trayectorias calculadas (en rojo) conectando las calles (azul) con los pozos.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
geofence.boundary.plot(ax=ax, color='green')
holes_filtered.plot(ax=ax, color='black', markersize=5)

# Plot street poses
x_street = [p[0] for p in poses_street]
y_street = [p[1] for p in poses_street]
ax.scatter(x_street, y_street, c='blue', s=2, label='Street Poses')

# Plot loading poses (trayectorias)
for hole_poses in poses_holes:
    if hole_poses:
        x_hole = [p[0] for p in hole_poses]
        y_hole = [p[1] for p in hole_poses]
        ax.plot(x_hole, y_hole, c='red', linewidth=1)

plt.title("Rutas Generadas (Detalle)")
plt.legend()
plt.show()